In [1]:
import numpy as np
import pandas as pd
import pseudopeople as psp

from pathlib import Path
import shutil
import os

In [ ]:
file_stem_ssa = "input_file_ssa"
file_stem_w2 = "input_file_w2"

years = [2020, 2030]

dfs = []
file_stems = []

for year in years:
    ssa = psp.generate_social_security(year=year)
    w2 = psp.generate_taxes_w2_and_1099(year=year)

    ssa["Record ID"] = ssa.index
    w2["Record ID"] = w2.index

    ssa = ssa[ssa["event_type"] == "creation"]

    # TODO: remove simulant_id column and save lookup table

    dfs.append(ssa)
    file_stems.append((file_stem_ssa, year))
    dfs.append(w2)
    file_stems.append((file_stem_w2, year))

display(dfs[0])
display(dfs[1])

,simulant_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,Record ID
0,0_19979,786-77-6454,Evelyn,Granddaughter,Sorrell,19191204,Female,creation,19191204,0
1,0_6846,688-88-6377,George,Robert,Kelly,19210616,Male,creation,19210616,1
2,0_19983,651-33-9561,Beatrice,Jennie,Livingston,19220113,Female,creation,19220113,2
3,0_262,665-25-7858,Eura,Nadine,Hutchison,19220305,Female,creation,19220305,3
4,0_12473,875-10-2359,Roberta,Ruth,Mcintyre,19220306,Female,creation,19220306,4
...,...,...,...,...,...,...,...,...,...,...
16492,0_20687,183-90-0619,Matthew,Michael,Stephens,19800224,Female,creation,20201229,16492
16493,0_20686,803-81-8527,Jermey,Tyler,Morris,19860415,Male,creation,20201229,16493
16494,0_20692,170-62-5253,Brittanie,Lauren,Kim,19950118,Female,creation,20201229,16494
16495,0_20662,281-88-9330,Marcus,Jasper,NaN,20201230,Male,creation,20201230,16495


,simulant_id,household_id,employer_id,ssn,wages,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,...,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,tax_form,tax_year,Record ID
0,0_4,0_8,95,584-16-0130,10192,Pikes Creek Campground,e,ince dr,NaN,Anytown,...,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2020,0
1,0_5,0_8,29,854-13-6295,28355,Red's Dairy Queen,264-12,hallmont dr,NaN,Anytown,...,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2020,1
2,0_5,0_8,30,854-13-6295,18243,Warrensburg,NaN,NaN,NaN,Anytown,...,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2020,2
3,0_5621,0_2289,46,674-27-1745,7704,Nashville City Properties,283,west id rd,NaN,Anytown,...,NaN,NaN,NaN,14011,Anytown,WA,00000,W2,2020,3
4,0_5623,0_2289,83,794-23-1522,3490,Ecr Whipple Oliver Finley Shoe Sensation,4682,buck tl,NaN,Anytown,...,NaN,NaN,NaN,14011,Anytown,WA,00000,W2,2020,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,0_18936,0_7621,23,006-92-7857,9585,Greenway Med Spa Services,631,regent ave,NaN,Anytown,...,512,w shadowbend ave,NaN,NaN,Anytown,WA,00000,W2,2020,9898
9899,0_18936,0_7621,90,006-92-7857,57906,Northwell Health Center Inc,NaN,barge roa sw,NaN,Anytown,...,512,w shadowbend ave,NaN,NaN,Anytown,WA,00000,W2,2020,9899
9900,0_18937,0_7621,1,182-82-5017,19609,Military,7930,moriah rd,NaN,Anytown,...,512,w shadowbend ave,NaN,NaN,Anytown,WA,00000,1099,2020,9900
9901,0_18937,0_7621,105,182-82-5017,8061,Waffle House of Big Daddys Burgers Shakes and ...,2329,greenwood wy,NaN,Anytown,...,512,w shadowbend ave,NaN,NaN,Anytown,WA,00000,1099,2020,9901


In [3]:
# save files to repo
for df in dfs:
    ssa.to_parquet(f"{file_stem_ssa}.parquet")
    w2.to_parquet(f"{file_stem_w2}.parquet")

In [4]:
for (stem, year), df in zip(file_stems, dfs):

    # save files to repo and copy to shared filesystem
    output_dir = Path(
        f"/mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/{os.getlogin()}/{year}"
    )
    if not output_dir.exists():
        # make the directory with rwxrwxr-x permissions
        output_dir.mkdir(parents=True, mode=0o775)

    local_dir = Path(f"{year}")
    if not local_dir.exists():
        # make the directory with rwxrwxr-x permissions
        local_dir.mkdir(parents=True, mode=0o775)

    filename = f"{stem}.parquet"
    df.to_parquet(local_dir / filename)
    filepath = output_dir / filename
    if filepath.exists():
        print(f"Overwriting existing file: {filepath}")
    shutil.copy(local_dir / filename, filepath)
    print(f"File copied to: {filepath}")

Overwriting existing file: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2020/input_file_ssa.parquet
File copied to: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2020/input_file_ssa.parquet
Overwriting existing file: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2020/input_file_w2.parquet
File copied to: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2020/input_file_w2.parquet


Overwriting existing file: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2030/input_file_ssa.parquet
File copied to: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2030/input_file_ssa.parquet
Overwriting existing file: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2030/input_file_w2.parquet
File copied to: /mnt/team/simulation_science/priv/engineering/er_ecosystem/input_data/tylerdy/2030/input_file_w2.parquet
